In [1]:
import pandas as pd

df = pd.read_csv('transformed.csv')
X = df.loc[:,'hour_of_day':'traffic_type_10']
Y = df.loc[:,'hits']
X.head()


,hour_of_day,session_durantion,path_length,path_imp,entry_page_imp,locale_L1,locale_L2,locale_L3,locale_L4,locale_L5,...,agent_id_13,agent_id_14,agent_id_15,traffic_type_1,traffic_type_2,traffic_type_3,traffic_type_4,traffic_type_6,traffic_type_7,traffic_type_10
0,22,49,2,0.442416,0.332617,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,21,1892,2,0.438970,0.178869,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,19,0,1,0.002216,0.332617,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6,2,2,0.438985,0.202408,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,1,0.438586,0.178869,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [2]:
!which python

/home/raghu/anaconda3/envs/TF-gpu/bin/python


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X,test_size=0.2,shuffle=False)

In [6]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib
scaler = StandardScaler(with_mean=False)
X_train.loc[:,'hour_of_day':'path_length'] = scaler.fit_transform(X_train.loc[:,'hour_of_day':'path_length'])
X_test.loc[:,'hour_of_day':'path_length'] = scaler.transform(X_test.loc[:,'hour_of_day':'path_length'])
X_scaled = np.concatenate((X_train,X_test))
joblib.dump(scaler,"scaler.pkl")
X_scaled


array([[3.24986178, 0.02512368, 2.0604232 , ..., 0.        , 0.        ,
        0.        ],
       [3.10214079, 0.97008161, 2.0604232 , ..., 0.        , 0.        ,
        0.        ],
       [2.80669881, 0.        , 1.0302116 , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [1.92037287, 0.31840416, 2.0604232 , ..., 0.        , 0.        ,
        0.        ],
       [0.73860495, 0.        , 2.0604232 , ..., 0.        , 0.        ,
        0.        ],
       [2.21581485, 2.32983659, 2.0604232 , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
def mlp(dropout_rate=0.0,activation1='relu', activation2='relu', activation3='relu', activation4='relu',
                       activation5='relu',activation6='relu'):
    model = Sequential()
    model.add(Dense(1024, input_dim=len(X.columns), kernel_initializer='normal', activation=activation1))
    model.add(Dropout(dropout_rate))
    model.add(Dense(512, kernel_initializer='normal', activation=activation2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, kernel_initializer='normal', activation=activation3))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, kernel_initializer='normal', activation=activation4))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, kernel_initializer='normal', activation=activation5))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, kernel_initializer='normal', activation=activation6))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

In [ ]:
import numpy as np
seed = 7
np.random.seed(seed)

In [ ]:
from keras.callbacks import  ModelCheckpoint, EarlyStopping, TensorBoard
outputFolder = '/media/raghu/6A3A-B7CD/PredictHits'
filepath = outputFolder + "/model-{val_loss:.4f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, \
                             save_best_only=True, save_weights_only=False, \
                             mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir='./logs')
callbacks_list = [earlystop, checkpoint, tensorboard]

In [ ]:
activation1 = ['elu','tanh','sigmoid']
activation2 = ['elu', 'sigmoid','tanh']
activation3 = ['elu', 'sigmoid','tanh']
activation4 = ['elu', 'sigmoid','tanh']
activation5 = ['elu', 'sigmoid','tanh']
activation6 = ['elu', 'tanh', 'sigmoid']
activation7 = ['relu']

dropout_rate = [0.2,0.3]

(row_size,_)=np.shape(X_train)
min_bsize=int(row_size/100)
batch_size=[min_bsize,2*min_bsize]

param_grid = dict(activation1=activation1, activation2=activation2, activation3=activation3, activation4=activation4,
                  activation5=activation5, activation6=activation6,  batch_size=batch_size, dropout_rate=dropout_rate)

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
estimator = KerasRegressor(build_fn=mlp, epochs=100,  verbose=2, validation_split=0.2,
                           callbacks=callbacks_list)


In [ ]:
from sklearn.model_selection import GridSearchCV,ShuffleSplit
grid = GridSearchCV(estimator, param_grid=param_grid, cv=ShuffleSplit(test_size=0.01, n_splits=1), scoring='neg_mean_absolute_error',verbose=2)

In [ ]:
# grid_result = grid.fit(X_scaled, Y.to_numpy())